In [31]:
import json
import numpy as np
import os

cw = os.getcwd()
pwd = os.path.abspath(os.path.join(cw, "..", "results"))
print(f"Parent working directory: {pwd}")

Parent working directory: /proj/sourasb-220503/IoT_attack_CL_IDS/results


In [36]:
import os, json
import numpy as np

def compute_plasticity_efficiency(
    files,
    ddof=0,
    eps=1e-5,
    print_table=True,
    kappa=3.0,          # ↑ increase to penalize plasticity more (e.g., 5.0)
    clamp_nonneg=True,  # clamp plasticity >= 0 so negative plasticity doesn't "help"
    weighted=True       # weight per-file means by counts
):
    """
    Plasticity per domain:  (after - before) from `performance_m` values.
    Accuracy per file:      mean of last values across keys in `performance_stability`.

    Penalized score (higher is better when stability/final accuracy dominates):
        PE_kappa = Abar / (Abar + kappa * Pbar + eps),  in [0, 1]
        where Pbar = mean plasticity (optionally clamped >= 0), Abar = mean accuracy.

    Prints a compact table and returns only an `overall` dict.
    """

    def get_plasticity_from_perf_m(perf_m):
        vals = []
        for v in (perf_m or {}).values():
            if isinstance(v, (list, tuple)) and len(v) >= 2:
                try:
                    vals.append(float(v[1]) - float(v[0]))
                except Exception:
                    pass
        return np.asarray(vals, dtype=float)

    def get_accuracy_from_perf_stability(perf_stab):
        vals = []
        for v in (perf_stab or {}).values():
            if isinstance(v, (list, tuple)) and len(v) >= 1:
                try:
                    vals.append(float(v[-1]))
                except Exception:
                    pass
        return np.asarray(vals, dtype=float)

    rows = []
    pooled_plast_list, pooled_acc_list = [], []

    for file in files:
        with open(file, "r") as f:
            data = json.load(f)

        plast_arr = get_plasticity_from_perf_m(data.get("performance_m"))
        acc_arr   = get_accuracy_from_perf_stability(data.get("performance_stability"))

        avg_plast = float(np.mean(plast_arr)) if plast_arr.size else np.nan
        std_plast = float(np.std(plast_arr, ddof=ddof)) if plast_arr.size else np.nan
        avg_acc   = float(np.mean(acc_arr))   if acc_arr.size   else np.nan
        std_acc   = float(np.std(acc_arr, ddof=ddof))   if acc_arr.size   else np.nan

        # --- Penalize plasticity (kappa * Pbar) ---
        if np.isnan(avg_plast):
            Pbar = np.nan
        else:
            Pbar = max(0.0, avg_plast) if clamp_nonneg else avg_plast

        Abar = avg_acc
        denom = (Abar + kappa * Pbar + eps) if (not np.isnan(Abar) and not np.isnan(Pbar)) else np.nan
        pe    = (Abar / denom) if not np.isnan(denom) else np.nan

        if plast_arr.size: pooled_plast_list.append(plast_arr)
        if acc_arr.size:   pooled_acc_list.append(acc_arr)

        rows.append((os.path.basename(file),
                     plast_arr.size, acc_arr.size,
                     avg_plast, std_plast, avg_acc, std_acc, pe))

    # ---- Compact table ----
    if print_table:
        header = f"{'File':<40} {'nP':>3} {'nA':>3} {'μP':>10} {'σP':>10} {'μA':>10} {'σA':>10} {'PEκ':>10}"
        print(header)
        print("-" * len(header))
        for fname, nP, nA, muP, sdP, muA, sdA, pe in rows:
            fmt = lambda x: ("NA" if np.isnan(x) else f"{x:10.6f}")
            print(f"{fname:<40} {nP:>3} {nA:>3} {fmt(muP)} {fmt(sdP)} {fmt(muA)} {fmt(sdA)} {fmt(pe)}")

    # ---- Overall (per-file means; optionally weighted by counts) ----
    muP = np.array([r[3] for r in rows], dtype=float)
    muA = np.array([r[5] for r in rows], dtype=float)
    pe  = np.array([r[7] for r in rows], dtype=float)
    nP  = np.array([r[1] for r in rows], dtype=float)
    nA  = np.array([r[2] for r in rows], dtype=float)

    def nnan_weighted_mean(x, w=None):
        m = ~np.isnan(x)
        if not m.any(): return np.nan
        if w is None: return float(np.mean(x[m]))
        w = np.array(w, dtype=float)[m]
        if w.sum() == 0: return float(np.mean(x[m]))
        return float(np.average(x[m], weights=w))

    def nnan_std(x):
        x = x[~np.isnan(x)]
        return float(np.std(x, ddof=ddof)) if x.size else np.nan

    wP = nP if weighted else None
    wA = nA if weighted else None

    overall_perfile_avg_plasticity = nnan_weighted_mean(muP, wP)
    overall_perfile_std_plasticity = nnan_std(muP)
    overall_perfile_avg_accuracy   = nnan_weighted_mean(muA, wA)
    overall_perfile_std_accuracy   = nnan_std(muA)
    overall_perfile_avg_efficiency = nnan_weighted_mean(pe, None)
    overall_perfile_std_efficiency = nnan_std(pe)

    # ---- Pooled across all domains ----
    pooled_plast = np.concatenate(pooled_plast_list) if pooled_plast_list else np.array([], dtype=float)
    pooled_acc   = np.concatenate(pooled_acc_list)   if pooled_acc_list   else np.array([], dtype=float)

    pooled_avg_plasticity = float(np.mean(pooled_plast)) if pooled_plast.size else np.nan
    pooled_std_plasticity = float(np.std(pooled_plast, ddof=ddof)) if pooled_plast.size else np.nan
    pooled_avg_accuracy   = float(np.mean(pooled_acc)) if pooled_acc.size else np.nan
    pooled_std_accuracy   = float(np.std(pooled_acc, ddof=ddof)) if pooled_acc.size else np.nan

    if np.isnan(pooled_avg_plasticity):
        Pbar_pool = np.nan
    else:
        Pbar_pool = max(0.0, pooled_avg_plasticity) if clamp_nonneg else pooled_avg_plasticity
    Abar_pool = pooled_avg_accuracy
    pooled_denom = (Abar_pool + kappa * Pbar_pool + eps) if (not np.isnan(Abar_pool) and not np.isnan(Pbar_pool)) else np.nan
    pooled_efficiency = (Abar_pool / pooled_denom) if not np.isnan(pooled_denom) else np.nan

    # ---- Minimal overall printout ----
    if print_table:
        def line(label, mean, std):
            m = "NA" if np.isnan(mean) else f"{mean:.6f}"
            s = "NA" if np.isnan(std)  else f"{std:.6f}"
            print(f"{label:<12} mean ± std = {m} ± {s}")

        print("\n===== Overall (per-file means) =====")
        line("Plasticity", overall_perfile_avg_plasticity, overall_perfile_std_plasticity)
        line("Accuracy",   overall_perfile_avg_accuracy,   overall_perfile_std_accuracy)
        line("PEκ",        overall_perfile_avg_efficiency, overall_perfile_std_efficiency)

        def singleton(label, val):
            v = "NA" if np.isnan(val) else f"{val:.6f}"
            print(f"{label:<12} {v}")
        print("\n===== Overall (pooled across domains) =====")
        singleton("μ Plasticity", Pbar_pool)
        singleton("σ Plasticity", pooled_std_plasticity)
        singleton("μ Accuracy",   Abar_pool)
        singleton("σ Accuracy",   pooled_std_accuracy)
        singleton("PEκ",          pooled_efficiency)

    


In [37]:
files_1 = [
    pwd + "/1_experiment_results_LSTM_WCL_random.json",
    pwd + "/2_experiment_results_LSTM_WCL_random.json",
    pwd + "/3_experiment_results_LSTM_WCL_random.json"
]

compute_plasticity_efficiency(files_1)


files_2 = [
    pwd + "/1_experiment_results_LSTM_WCL_b2w.json",
    pwd + "/2_experiment_results_LSTM_WCL_b2w.json",
    pwd + "/3_experiment_results_LSTM_WCL_b2w.json"
]

compute_plasticity_efficiency(files_2)


files_3 = [
    pwd + "/1_experiment_results_LSTM_WCL_w2b.json",
    pwd + "/2_experiment_results_LSTM_WCL_w2b.json",
    pwd + "/3_experiment_results_LSTM_WCL_w2b.json"
]

compute_plasticity_efficiency(files_3)


files_4 = [
    pwd + "/1_experiment_results_LSTM_WCL_toggle.json",
    pwd + "/2_experiment_results_LSTM_WCL_toggle.json",
    pwd + "/3_experiment_results_LSTM_WCL_toggle.json"
]

compute_plasticity_efficiency(files_4)


File                                      nP  nA         μP         σP         μA         σA        PEκ
-------------------------------------------------------------------------------------------------------
1_experiment_results_LSTM_WCL_random.json  47  48   0.248722   0.229413   0.474636   0.209114   0.388786
2_experiment_results_LSTM_WCL_random.json  47  48   0.257262   0.233432   0.501525   0.195858   0.393872
3_experiment_results_LSTM_WCL_random.json  47  48   0.257060   0.231021   0.496155   0.195319   0.391492

===== Overall (per-file means) =====
Plasticity   mean ± std = 0.254348 ± 0.003979
Accuracy     mean ± std = 0.490772 ± 0.011619
PEκ          mean ± std = 0.391384 ± 0.002078

===== Overall (pooled across domains) =====
μ Plasticity 0.254348
σ Plasticity 0.231329
μ Accuracy   0.490772
σ Accuracy   0.200535
PEκ          0.391420
File                                      nP  nA         μP         σP         μA         σA        PEκ
------------------------------------------

In [38]:
files_1 = [
    pwd + "/1_experiment_results_LSTM_EWC_random.json",
    pwd + "/2_experiment_results_LSTM_EWC_random.json",
    pwd + "/3_experiment_results_LSTM_EWC_random.json"
]

compute_plasticity_efficiency(files_1)


files_2 = [
    pwd + "/1_experiment_results_LSTM_EWC_b2w.json",
    pwd + "/2_experiment_results_LSTM_EWC_b2w.json",
    pwd + "/3_experiment_results_LSTM_EWC_b2w.json"
]

compute_plasticity_efficiency(files_2)


files_3 = [
    pwd + "/1_experiment_results_LSTM_EWC_w2b.json",
    pwd + "/2_experiment_results_LSTM_EWC_w2b.json",
    pwd + "/3_experiment_results_LSTM_EWC_w2b.json"
]

compute_plasticity_efficiency(files_3)


files_4 = [
    pwd + "/1_experiment_results_LSTM_EWC_toggle.json",
    pwd + "/2_experiment_results_LSTM_EWC_toggle.json",
    pwd + "/3_experiment_results_LSTM_EWC_toggle.json"
]

compute_plasticity_efficiency(files_4)


File                                      nP  nA         μP         σP         μA         σA        PEκ
-------------------------------------------------------------------------------------------------------
1_experiment_results_LSTM_EWC_random.json  47  48   0.050463   0.107752   0.571708   0.219709   0.790627
2_experiment_results_LSTM_EWC_random.json  47  48   0.111720   0.202517   0.594428   0.256674   0.639446
3_experiment_results_LSTM_EWC_random.json  47  48   0.074252   0.132145   0.575884   0.218071   0.721071

===== Overall (per-file means) =====
Plasticity   mean ± std = 0.078812 ± 0.025215
Accuracy     mean ± std = 0.580673 ± 0.009874
PEκ          mean ± std = 0.717048 ± 0.061785

===== Overall (pooled across domains) =====
μ Plasticity 0.078812
σ Plasticity 0.154912
μ Accuracy   0.580673
σ Accuracy   0.232380
PEκ          0.710635
File                                      nP  nA         μP         σP         μA         σA        PEκ
------------------------------------------

In [39]:
files_1 = [
    pwd + "/1_experiment_results_LSTM_SI_random.json",
    pwd + "/2_experiment_results_LSTM_SI_random.json",
    pwd + "/3_experiment_results_LSTM_SI_random.json"
]

compute_plasticity_efficiency(files_1)


files_2 = [
    pwd + "/1_experiment_results_LSTM_SI_b2w.json",
    pwd + "/2_experiment_results_LSTM_SI_b2w.json",
    pwd + "/3_experiment_results_LSTM_SI_b2w.json"
]

compute_plasticity_efficiency(files_2)


files_3 = [
    pwd + "/1_experiment_results_LSTM_SI_w2b.json",
    pwd + "/2_experiment_results_LSTM_SI_w2b.json",
    pwd + "/3_experiment_results_LSTM_SI_w2b.json"
]

compute_plasticity_efficiency(files_3)


files_4 = [
    pwd + "/1_experiment_results_LSTM_SI_toggle.json",
    pwd + "/2_experiment_results_LSTM_SI_toggle.json",
    pwd + "/3_experiment_results_LSTM_SI_toggle.json"
]

compute_plasticity_efficiency(files_4)

File                                      nP  nA         μP         σP         μA         σA        PEκ
-------------------------------------------------------------------------------------------------------
1_experiment_results_LSTM_SI_random.json  47  48   0.024337   0.069931   0.550765   0.181511   0.882941
2_experiment_results_LSTM_SI_random.json  47  48   0.027811   0.088343   0.533930   0.204498   0.864841
3_experiment_results_LSTM_SI_random.json  47  48   0.032399   0.084884   0.550564   0.182038   0.849936

===== Overall (per-file means) =====
Plasticity   mean ± std = 0.028182 ± 0.003302
Accuracy     mean ± std = 0.545086 ± 0.007889
PEκ          mean ± std = 0.865906 ± 0.013495

===== Overall (pooled across domains) =====
μ Plasticity 0.028182
σ Plasticity 0.081512
μ Accuracy   0.545086
σ Accuracy   0.189816
PEκ          0.865707
File                                      nP  nA         μP         σP         μA         σA        PEκ
---------------------------------------------

In [40]:

files_1 = [
    pwd + "/1_experiment_results_LSTM_Replay_REPLAY_random.json",
    pwd + "/2_experiment_results_LSTM_Replay_REPLAY_random.json",
    pwd + "/3_experiment_results_LSTM_Replay_REPLAY_random.json"
]

compute_plasticity_efficiency(files_1)


files_2 = [
    pwd + "/1_experiment_results_LSTM_Replay_REPLAY_b2w.json",
    pwd + "/2_experiment_results_LSTM_Replay_REPLAY_b2w.json",
    pwd + "/3_experiment_results_LSTM_Replay_REPLAY_b2w.json"
]

compute_plasticity_efficiency(files_2)


files_3 = [
    pwd + "/1_experiment_results_LSTM_Replay_REPLAY_w2b.json",
    pwd + "/2_experiment_results_LSTM_Replay_REPLAY_w2b.json",
    pwd + "/3_experiment_results_LSTM_Replay_REPLAY_w2b.json"
]

compute_plasticity_efficiency(files_3)


files_4 = [
    pwd + "/1_experiment_results_LSTM_Replay_REPLAY_toggle.json",
    pwd + "/2_experiment_results_LSTM_Replay_REPLAY_toggle.json",
    pwd + "/3_experiment_results_LSTM_Replay_REPLAY_toggle.json"
]

compute_plasticity_efficiency(files_4)


File                                      nP  nA         μP         σP         μA         σA        PEκ
-------------------------------------------------------------------------------------------------------
1_experiment_results_LSTM_Replay_REPLAY_random.json  47  48   0.305970   0.246600   0.783898   0.177653   0.460624
2_experiment_results_LSTM_Replay_REPLAY_random.json  47  48   0.311454   0.238861   0.764292   0.186148   0.449937
3_experiment_results_LSTM_Replay_REPLAY_random.json  47  48   0.327588   0.227875   0.770886   0.165482   0.439587

===== Overall (per-file means) =====
Plasticity   mean ± std = 0.315004 ± 0.009176
Accuracy     mean ± std = 0.773025 ± 0.008146
PEκ          mean ± std = 0.450049 ± 0.008589

===== Overall (pooled across domains) =====
μ Plasticity 0.315004
σ Plasticity 0.238080
μ Accuracy   0.773025
σ Accuracy   0.176819
PEκ          0.449944
File                                      nP  nA         μP         σP         μA         σA        PEκ
------------

In [41]:
files_1 = [
    pwd + "/1_experiment_results_LSTM_GR_random.json",
    pwd + "/2_experiment_results_LSTM_GR_random.json",
    pwd + "/3_experiment_results_LSTM_GR_random.json"
]

compute_plasticity_efficiency(files_1)

files_2 = [
    pwd + "/1_experiment_results_LSTM_GR_b2w.json",
    pwd + "/2_experiment_results_LSTM_GR_b2w.json",
    pwd + "/3_experiment_results_LSTM_GR_b2w.json"
]

compute_plasticity_efficiency(files_2)

files_3 = [
    pwd + "/1_experiment_results_LSTM_GR_w2b.json",
    pwd + "/2_experiment_results_LSTM_GR_w2b.json",
    pwd + "/3_experiment_results_LSTM_GR_w2b.json"
]
compute_plasticity_efficiency(files_3)

files_4 = [
    pwd + "/1_experiment_results_LSTM_GR_toggle.json",
    pwd + "/2_experiment_results_LSTM_GR_toggle.json",
    pwd + "/3_experiment_results_LSTM_GR_toggle.json"
]
compute_plasticity_efficiency(files_4)



File                                      nP  nA         μP         σP         μA         σA        PEκ
-------------------------------------------------------------------------------------------------------
1_experiment_results_LSTM_GR_random.json  47  48   0.247555   0.222755   0.501884   0.203488   0.403263
2_experiment_results_LSTM_GR_random.json  47  48   0.226901   0.214892   0.450131   0.188350   0.398049
3_experiment_results_LSTM_GR_random.json  47  48   0.257766   0.215212   0.515254   0.221129   0.399867

===== Overall (per-file means) =====
Plasticity   mean ± std = 0.244074 ± 0.012839
Accuracy     mean ± std = 0.489090 ± 0.028084
PEκ          mean ± std = 0.400393 ± 0.002161

===== Overall (pooled across domains) =====
μ Plasticity 0.244074
σ Plasticity 0.218028
μ Accuracy   0.489090
σ Accuracy   0.206678
PEκ          0.400459
File                                      nP  nA         μP         σP         μA         σA        PEκ
---------------------------------------------